In [2]:
# loading library
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

#size - in pixels, size of motion blur
#angel - in degrees, direction of motion blur
def motion_blur(image, size, angle):
    k = np.zeros((size, size), dtype=np.float32)
    k[ (size-1)// 2 , :] = np.ones(size, dtype=np.float32)
    rotation_matrix = cv.getRotationMatrix2D( (size / 2 -0.5 , size / 2 -0.5 ) , angle, 1.0)
    k = cv.warpAffine(k, rotation_matrix, (size, size) )
    k = k * ( 1.0 / np.sum(k) )
    return cv.filter2D(image, -1, k)

def random_motion_blur(image, size=(0, 5)):
    deg = 360 * np.random.random_sample() # random angle from 0 to 360
    rad = np.random.randint(*size)        # random size within given interval
    return motion_blur(image, rad, deg)

def gaussian_blur(image, size=5):
    return cv.GaussianBlur(image, (size, size), 0)


In [3]:
def increase_brightness(image, value):
    hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    h, s, v = cv.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv.merge((h, s, v))
    image = cv.cvtColor(final_hsv, cv.COLOR_HSV2BGR)
    return image

def decrease_brightness(image, value):
    hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    h, s, v = cv.split(hsv)

    v[v <= value] = 0
    v[v >  value] -= value

    final_hsv = cv.merge((h, s, v))
    image = cv.cvtColor(final_hsv, cv.COLOR_HSV2BGR)
    return image

def adjust_brightness(image, value):
    return decrease_brightness(image, -1*value) if value < 0 else increase_brightness(image, value)

def random_brightness(image, size=(-30, 30)):
    adj = np.random.randint(*size) # random brightness adjustment
    return adjust_brightness(image, adj)

In [4]:
def create_effect(fn, *args, **kwargs):
    return lambda img : fn(img, *args, **kwargs)

def apply_effects(img, fxs):
    for fx in fxs:
        img = fx(img)
    return img

In [8]:
import os

def generate_image_with_effects(in_dir, out_dir, fxs, pref=""):
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
        
    for file in os.listdir(in_dir):
        in_path = os.path.join(in_dir, file)
        out_path = os.path.join(out_dir, pref + file)
        
        img = cv.imread(in_path)
        img = apply_effects(img, fxs)
        
        cv.imwrite(out_path, img)
        
def show_three(src):
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12,12), subplot_kw={'xticks': [], 'yticks': []})
    for i, ax in enumerate(axes.flat):
        ax.imshow(plt.imread(f"{src}/0000{i+1}.jpg"))

    plt.tight_layout()
    plt.show()
    
# img = cv.imread('./images/Positive/00001.jpg')

# dec = create_effect(decrease_brightness, 14)
# gau = create_effect(gaussian_blur,       17)

# img = apply_effects(img, [dec, gau])

# plt.imshow(random_motion_blur(img, (5, 50)))

In [10]:
# random_brightness_effect = create_effect(random_brightness, size=(-80, 80))
random_motion_blur_effect = create_effect(random_motion_blur, size=(1, 8))
# generate_image_with_effects("./Utah/C", "./Utah/C", [random_brightness_effect], pref="aa")
generate_image_with_effects("./Utah/C", "./Utah/C", [random_motion_blur_effect], pref="ab")

In [19]:
tn = 545
tp = 18900
fn = 20000 - tp
fp = 20000 - tn

tn /= 40000
tp /= 40000
fn /= 40000
fp /= 40000

# calculate the accuracy based on the number of true positives and true negatives
accuracy = (tp + tn) / (tp + tn + fp + fn)
# calculate the precision based on the number of true positives and false positives
precision = tp / (tp + fp)
# calculate the recall based on the number of true positives and the number of positives
recall = tp / (tp + fn)
# calculate the f1 score based on the precision and the recall
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"Accuracy: {round(accuracy, 5)}")
print(f"Precision: {round(precision, 5)}")
print(f"Recall: {round(recall, 5)}")
print(f"F1 Score: {round(f1_score, 5)}")

Accuracy: 0.48613
Precision: 0.49276
Recall: 0.945
F1 Score: 0.64776
